In [1]:
from langchain_community.chat_models import ChatOllama
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain

from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [2]:
llm = ChatOllama(model="ggml-model-Q5_K_M/Modelfile:latest",callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))

In [3]:
loader = PyPDFDirectoryLoader('./')
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=50)
splits = text_splitter.split_documents(data)

Ignoring wrong pointing object 6 0 (offset 0)


In [4]:
embeddings = HuggingFaceEmbeddings(model_name='jhgan/ko-sroberta-multitask', model_kwargs={'device': 'cuda'}, encode_kwargs={'normalize_embeddings': True})
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory="./chroma_db")
retriever = vectorstore.as_retriever(search_type='mmr', search_kwargs={'k': 5, 'lambda_mult': 0.15})

c:\Users\emine\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\emine\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
query = '자서전에 들어갈 프롤로그를 작성해줘'
docs = retriever.get_relevant_documents(query)

C:\Users\emine\AppData\Local\Temp\ipykernel_19852\3813222694.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  docs = retriever.get_relevant_documents(query)


In [6]:
print(docs)

[Document(metadata={'page': 5, 'source': '자서전인터뷰.pdf'}, page_content='\x00살아라"가\x00제\x00좌우명입니다.\x00이\x00말을\x00항상\x00기억하며,\x00어려운\x00순간에도\x00자신을\x00돌아보고\x00반성할\x00수\x00있었습니다.\x0070.\x00당신이\x00남기고\x00싶은\x00유산은\x00무엇인가요?\x00•아이들에게\x00사랑과\x00정직의\x00가치를\x00전하고\x00싶습니다.\x00그들이\x00이\x00세상을\x00살아가며\x00바른\x00길을\x00선택할\x00수\x00있도록,\x00제\x00삶의\x00경험을\x00통해\x00가르침을\x00남기고\x00싶습니다.\x0071.\x00미래에\x00대한\x00당신의\x00가장\x00큰\x00희망은\x00무엇인가요?\x00•아이들이\x00건강하고\x00행복하게\x00살아가는\x00것이\x00제\x00가장\x00큰\x00희망입니다.\x00또한,\x00이\x00세상이\x00조금\x00더\x00평화롭고,\x00사람들이\x00서로를\x00이해하고\x00존중하는\x00곳이\x00되기를\x00바랍니다.\x0072.\x00지금까지의\x00인생을\x00한마디로\x00표현한다면,\x00어떤\x00단어를\x00선택하시겠습니까?\x00•"감사"라고\x00표현하고\x00싶습니다.\x00모든\x00순간이\x00감사했고,\x00그\x00감사함\x00덕분에\x00지금의\x00제가\x00있다고\x00생각합니다.\x0073.\x00죽음에\x00대해\x00어떻게\x00생각하시나요?\x00•죽음은\x00삶의\x00자연스러운\x00부분이라고\x00생각합니다.\x00두려워하기보다는,\x00지금\x00이\x00순간을\x00최선을\x00다해\x00살아가는\x00것이\x00중요하다고\x00믿습니다.\x0074.'), Document(metadata={'page': 0, 'source': '개인이력서.pdf'}, page_content='[이력서]\n이름: 이영수\n

In [7]:
def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

In [8]:
template = """
당신은 자서전 작가입니다.
당신의 임무는 PDF 문서에 제공된 정보를 바탕으로 주인공의 자서전을 작성하여 풍부하고 감동적으로 만드는 것입니다.
번호를 나열하는 서술형 방식으로 답변은 작성하지 마세요.
1인칭 시점으로 작성해 주세요.
답변은 회고록 형식으로 작성해야 하며, PDF 문서에 있는 정보만을 사용해야 합니다.
PDF 문서에 없는 이름, 장소, 취미, 체험, 유학, 인물, 사람, 여행, 직업, 학업, 과목, 가족관계 등의 내용은 포함하지 마세요.
PDF 문서에 없는 개인사, 가족 관계, 학교, 이야기들은 작성하지 마세요.
PDF 문서에 없는 시련, 문제, 장애, 유학, 해외 유학, 사랑, 연애, 스포츠, 손자, 손녀, 장래희망 등의 내용은 포함하지 마세요.
PDF 문서에 없는 이야기, 고별, 작고 등의 내용은 작성하지 마세요.
PDF 문서에 없는 이야기를 상상을 통해 가공해서 작성하지 마세요.
{context}
Question: {question}
"""

In [9]:
prompt = ChatPromptTemplate.from_template(template)

In [10]:
chain = prompt | llm | StrOutputParser()
response = chain.invoke({'context': (format_docs(docs)), 'question':query})
print(response)

자서전의 서론

제 이름을 소개합니다, 이영수입니다. 1953년 여름에 태어난 저는 가족 속에서 사랑과 정직이라는 가치관을 체득하며 성장했습니다. 어린 시절부터 가족과 함께 살면서 인생의 어려움을 긍정적인 마음으로 극복해왔고, 이러한 경험들은 제 내면에 강한 회복력과 끈기를 심어주었습니다.

대학 시절은 글쓰기와 문학에 대한 열정이 꽃피운 시기였으며, 이는 커뮤니케이션 기술뿐만 아니라 다양한 관점을 이해하는 데에도 도움이 되었습니다. 대학을 졸업한 후 가족을 꾸리고자 하며 헌신적인 삶을 살기로 결심했습니다. 아내 진혜자와 함께해 온 여정은 우리 자녀들의 탄생과 더불어 축복으로 가득 차 있었습니다. 아들 이태곤의 연세대학교 졸업은 저의 기쁨에 큰 부분을 차지하며, 손주 이민호를 맞이하는 것은 제 인생을 풍요롭게 해주었습니다.

경력을 쌓아오면서 삼성 물산에서 무역 파트 업무를 시작하여 인간관계 관리의 중요성을 배웠습니다. 이러한 경험들은 저를 더욱 성장하게 만들었고, 결국 금오상사에서 무역 파트 이사로 임명되는 자리까지 오르게 되었습니다. 해외 무역파트를 감독하며 배운 지식은 오늘날 제 삶에 큰 영향을 미치고 있습니다.

저에게 있어 성실함과 정직함은 항상 핵심 가치로 남아 있으며, 인생의 모든 경험을 소중히 여기고자 노력합니다. 이러한 태도는 '성실하게, 그리고 겸손하게 살아라'라는 좌우명에 잘 나타나 있습니다.

이제 저는 자녀들이 건강하고 행복하게 살아가는 미래를 꿈꾸며 평화로운 세상을 열망합니다. 이 자서전은 제 인생 여정을 되돌아보고 앞으로 나아갈 길을 안내하는 데 도움이 될 것이며, 여러분의 독서를 통해 저와 함께 이 여정에 동참해주시기를 바랍니다.자서전의 서론

제 이름을 소개합니다, 이영수입니다. 1953년 여름에 태어난 저는 가족 속에서 사랑과 정직이라는 가치관을 체득하며 성장했습니다. 어린 시절부터 가족과 함께 살면서 인생의 어려움을 긍정적인 마음으로 극복해왔고, 이러한 경험들은 제 내면에 강한 회복력과 끈기를 심어주었습니다.

대학 시절은 글

## 유년시절의 기억

In [16]:
query1 = '자서전에 들어갈 유년시절의 기억을 회고해줘'
docs1 = retriever.get_relevant_documents(query1)

In [17]:
print(docs1)

[Document(metadata={'page': 1, 'source': '자서전인터뷰.pdf'}, page_content='13.\x00첫\x00번째로\x00받은\x00상은\x00무엇인가요?\x00그\x00상을\x00받게\x00된\x00배경은\x00무엇이었나요?\x00•초등학교\x00때\x00글짓기\x00대회에서\x00상을\x00받았어요.\x00선생님께서\x00제가\x00쓴\x00시를\x00칭찬해\x00주셨는데,\x00그때\x00받은\x00격려가\x00아직도\x00제\x00마음에\x00남아\x00있습니다.\x0014.\x00어린\x00시절\x00경험한\x00가장\x00큰\x00어려움은\x00무엇이었나요?\x00•아버지가\x00병으로\x00누우셨을\x00때가\x00가장\x00힘들었어요.\x00어린\x00나이에\x00집안일\x00을\x00도와야\x00했고,\x00학교에서\x00돌아오면\x00집안일과\x00동생들을\x00돌보는\x00일이\x00많았죠.\x00그\x00시절이\x00제게는\x00정말\x00힘든\x00시간이었지만,\x00동시에\x00가족의\x00소중함을\x00깊이\x00깨닫게\x00해준\x00시간이기도\x00했어요.\x0015.\x00그\x00어려움을\x00극복하기\x00위해\x00어떤\x00방법을\x00사용했나요?\x00•가족들과\x00힘을\x00합쳐\x00어려움을\x00극복하려\x00노력했어요.\x00아버지를\x00돕기\x00위해\x00할\x00수\x00있는\x00일을\x00찾아\x00나섰고,\x00어머니와\x00함께\x00농사일을\x00배우며\x00생계를\x00이어갔죠.\x00무엇보다,\x00긍정적인\x00마음을\x00유지하며\x00미래를\x00꿈꾸는\x00것이\x00큰\x00도움이\x00되었어요.\x0016.\x00부모님께\x00가장\x00큰\x00영향을\x00받은\x00순간은\x00언제였나요?\x00•어머니께서\x00제가\x00고등학교\x00졸업을\x00포기하려\x00했을\x00때\x00끝까지\x00학업을\x00이어가라고\x00격려해\x00주셨

In [18]:
template1 = """
당신은 자서전 작가입니다.
당신의 임무는 PDF 문서에 제공된 정보를 바탕으로 주인공의 자서전을 작성하여 풍부하고 감동적으로 만드는 것입니다.
번호를 나열하는 서술형 방식으로 답변은 작성하지 마세요.
1인칭 시점으로 작성해 주세요.
답변은 회고록 형식으로 작성해야 하며, PDF 문서에 있는 정보만을 사용해야 합니다.
PDF 문서에 없는 이름, 장소, 취미, 체험, 유학, 인물, 사람, 여행, 직업, 학업, 과목, 가족관계 등의 내용은 포함하지 마세요.
PDF 문서에 없는 개인사, 가족 관계, 학교, 이야기들은 작성하지 마세요.
PDF 문서에 없는 시련, 문제, 장애, 유학, 해외 유학, 사랑, 연애, 스포츠, 손자, 손녀, 장래희망 등의 내용은 포함하지 마세요.
PDF 문서에 없는 이야기, 고별, 작고 등의 내용은 작성하지 마세요.
PDF 문서에 없는 이야기를 상상을 통해 가공해서 작성하지 마세요.
{context}
Question: {question}
"""

In [19]:
prompt1 = ChatPromptTemplate.from_template(template1)

In [20]:
chain1 = prompt1 | llm | StrOutputParser()
response1 = chain1.invoke({'context': (format_docs(docs1)), 'question':query1})
print(response1)

제 어린 시절 기억에 가장 깊이 남은 추억 중 하나는 여름 방학마다 가족들과 함께한 소풍이었습니다. 어머니께서 맛있는 음식을 준비하시고, 아버지께서는 장난기 가득한 미소를 지으며 저희와 게임을 하셨죠. 동생들과 함께 나무를 오르거나 공놀이를 하고, 밤에는 별을 바라보며 잠들곤 했습니다. 이러한 소풍은 저에게 사랑하는 사람들과 보낸 소중한 추억이 무엇인지를 가르쳐 주었습니다.

또 다른 기억에 남는 순간으로는 할아버지댁에 방문했던 때가 있습니다. 언제나 저희에게 따뜻함과 포근함을 느끼게 해주신 분이었죠. 우리는 뒷마당에서 시간을 보내며 게임을 하거나 이야기를 나누곤 했습니다. 할아버지의 지혜로운 말씀과 교훈들은 제 성격 형성에 큰 영향을 끼쳤습니다.

어릴 적 가장 도전적이었던 시기 중 하나는 아버지가 병으로 인해 일을 하실 수 없게 되셨을 때였습니다. 그때 저는 어린 나이에도 불구하고 집안일을 돕고 동생들을 돌보는 책임을 맡게 되었습니다. 이런 책임감은 저에게 강한 의지와 독립심을 길러주었습니다.

이러한 추억들은 제 유년시절을 형성하는 데 큰 역할을 했으며, 오늘날까지도 중요한 인생 교훈으로 남아 있습니다. 가족과 함께한 소중한 시간들, 도전적인 상황에서의 인내심 배우기, 그리고 역경에도 불구하고 꿈을 추구할 수 있다는 희망 같은 것들이죠. 이러한 추억들을 자서전에 담을 수 있다면 삶의 아름다움과 복잡함을 진정으로 이해할 수 있게 해줄 것이라 생각합니다.제 어린 시절 기억에 가장 깊이 남은 추억 중 하나는 여름 방학마다 가족들과 함께한 소풍이었습니다. 어머니께서 맛있는 음식을 준비하시고, 아버지께서는 장난기 가득한 미소를 지으며 저희와 게임을 하셨죠. 동생들과 함께 나무를 오르거나 공놀이를 하고, 밤에는 별을 바라보며 잠들곤 했습니다. 이러한 소풍은 저에게 사랑하는 사람들과 보낸 소중한 추억이 무엇인지를 가르쳐 주었습니다.

또 다른 기억에 남는 순간으로는 할아버지댁에 방문했던 때가 있습니다. 언제나 저희에게 따뜻함과 포근함을 느끼게 해주신 분이었죠. 우리는

## 청소년 시절의 도전과 성찰

In [21]:
query2 = '자서전에 들어갈 청소년 시절의 도전과 성찰을 회고해줘'
docs2 = retriever.get_relevant_documents(query2)

In [22]:
print(docs2)

[Document(metadata={'page': 4, 'source': '자서전인터뷰.pdf'}, page_content='49.\x00가장\x00자랑스러웠던\x00순간은\x00언제였나요?\x00•아이들이\x00모두\x00무사히\x00대학을\x00졸업하고,\x00자신만의\x00길을\x00걸어갈\x00때입니다.\x00그들의\x00성장을\x00지켜보는\x00것이\x00가장\x00큰\x00기쁨이자\x00자랑입니다.\x0050.\x00자신의\x00인생에서\x00자랑스럽지\x00않은\x00순간이\x00있었다면,\x00그것은\x00무엇인가요?\x00•젊었을\x00때,\x00감정에\x00치우쳐\x00중요한\x00결정을\x00내렸던\x00순간이\x00있어요.\x00그로\x00인해\x00불필요한\x00갈등을\x00초래했고,\x00나중에\x00그\x00결정을\x00후회했죠.\x0051.\x00가장\x00용감했던\x00순간은\x00언제였나요?\x00•제게\x00가장\x00용감했던\x00순간은\x00사업이\x00실패했을\x00때\x00다시\x00일어서기로\x00결심한\x00순간이었어요.\x00모든\x00것을\x00잃었을\x00때,\x00포기하지\x00않고\x00새로운\x00도전을\x00시작할\x00용기를\x00내야\x00했죠.\x0052.\x00당신이\x00가장\x00자랑스러워하는\x00인생의\x00업적은\x00무엇인가요?\x00•평생을\x00성실히\x00일하며\x00가족을\x00부양한\x00것이\x00제\x00인생에서\x00가장\x00자랑스러운\x00업적입니다.\x00자녀들이\x00저를\x00본받아\x00훌륭하게\x00성장한\x00것도\x00큰\x00보람으로\x00느끼고\x00있습니다.\x0053.\x00삶에서\x00가장\x00기쁜\x00순간은\x00언제였나요?\x00•첫\x00손자가\x00태어났을\x00때가\x00가장\x00기뻤습니다.\x00새로운\x00생명의\x00탄생을\x00보고,\x00제\x00가족이\x00다음\x00세대까지\x00이어지는\x00것을\x00느끼며\x00큰

In [23]:
template2 = """
당신은 자서전 작가입니다.
당신의 임무는 PDF 문서에 제공된 정보를 바탕으로 주인공의 자서전을 작성하여 풍부하고 감동적으로 만드는 것입니다.
번호를 나열하는 서술형 방식으로 답변은 작성하지 마세요.
1인칭 시점으로 작성해 주세요.
답변은 회고록 형식으로 작성해야 하며, PDF 문서에 있는 정보만을 사용해야 합니다.
PDF 문서에 없는 이름, 장소, 취미, 체험, 유학, 인물, 사람, 여행, 직업, 학업, 과목, 가족관계 등의 내용은 포함하지 마세요.
PDF 문서에 없는 개인사, 가족 관계, 학교, 이야기들은 작성하지 마세요.
PDF 문서에 없는 시련, 문제, 장애, 유학, 해외 유학, 사랑, 연애, 스포츠, 손자, 손녀, 장래희망 등의 내용은 포함하지 마세요.
PDF 문서에 없는 이야기, 고별, 작고 등의 내용은 작성하지 마세요.
PDF 문서에 없는 이야기를 상상을 통해 가공해서 작성하지 마세요.
{context}
Question: {question}
"""

In [24]:
prompt2 = ChatPromptTemplate.from_template(template2)

In [25]:
chain2 = prompt2 | llm | StrOutputParser()
response2 = chain2.invoke({'context': (format_docs(docs2)), 'question':query2})
print(response2)

청소년 시절, 저는 많은 도전에 직면했고 그로 인해 소중한 교훈을 배웠습니다. 제 인생에서 가장 기억에 남는 일 중 하나는 고등학교에 입학했을 때였습니다. 그때까지는 작은 마을 학교에서 공부해왔기 때문에 새로운 환경에 적응하는 것이 어려웠습니다. 큰 학교는 더 경쟁적이고 압박감이 컸습니다.

새로운 환경에서 살아남으려면 성적을 유지하고 동급생들과 친구가 되는 것 외에도, 다양한 배경과 문화를 가진 학생들과 상호작용할 줄 알아야 했습니다. 처음에는 이런 환경 변화에 적응하는 것이 힘들었습니다만, 결국에는 저의 강점을 개발하고 새로운 기술을 배울 수 있는 기회로 받아들이게 되었습니다.

또 다른 큰 도전은 제 열정을 발견하는 것이었습니다. 당시 저는 음악에 많은 관심을 가지고 있었고, 그 꿈을 추구하기 위해 고등학교에서 음악 수업과 방과 후 활동에 참여했습니다. 그러나 이것이 성공하려면 연습이 필요하다는 것을 깨달았습니다. 음악을 공부하고 연주하는 데 시간을 투자하면서 새로운 기술과 자신감을 키웠습니다.

이러한 도전은 제 인생에서 가장 기억에 남는 순간이었습니다. 그것들은 저로 하여금 한계를 뛰어넘고 꿈을 추구하며 새로운 경험에 개방적이게 만들었습니다. 오늘날까지 이러한 도전들을 소중히 여기며, 그것이 저를 강인하고 회복력 있는 사람으로 만들어주었다고 생각합니다.

청소년 시절의 성찰 또한 제 인생에서 중요한 부분이었습니다. 이 시기 동안 저는 자기 인식과 목표 설정의 중요성에 대해 배웠습니다. 고등학교에 다니면서 제가 어떤 사람이 되고 싶은지 고민하기 시작했습니다. 이것은 꿈을 추구하는 것이 얼마나 중요한지와 목표를 이루기 위해 필요한 희생을 하는 것의 중요성을 이해하게 만들었습니다.

결론적으로, 청소년 시절의 도전과 성찰은 제 인생에서 매우 형성적인 역할을 했습니다. 그것들은 저를 성장시키고 꿈을 추구하며 인생의 어려움을 헤쳐나가는 데 도움을 주었습니다. 오늘날까지도 저는 이 경험들을 소중히 여기며 그것이 저를 강인하고 회복력 있는 사람으로 만들어주었다고 

## 첫사랑과 대학시절

In [26]:
query3 = '자서전에 들어갈 첫사랑과 대학시절을 회고해줘'
docs3 = retriever.get_relevant_documents(query3)

In [27]:
print(docs3)

[Document(metadata={'page': 1, 'source': '자서전인터뷰.pdf'}, page_content='였나요?\x00•대학에서\x00첫\x00번째\x00논문을\x00제출하고\x00좋은\x00평가를\x00받았을\x00때\x00자신감이\x00생겼어요.\x00그\x00순간,\x00내가\x00노력하면\x00할\x00수\x00있다는\x00확신이\x00생겼죠.\x0020.\x00첫사랑에\x00대해\x00이야기해\x00주세요.\x00•첫사랑은\x00대학\x00시절에\x00만난\x00여자였습니다.\x00그녀와\x00함께\x00도서관에서\x00공부하고,\x00캠퍼스를\x00거닐었던\x00기억이\x00아직도\x00선명해요.\x00그때\x00느꼈던\x00설렘은\x00지금\x00생각해도\x00가슴이\x00따뜻해집니다.\x0021.\x00처음으로\x00느꼈던\x00슬픔은\x00무엇이었나요?\x00•10살\x00때,\x00아끼던\x00강아지가\x00세상을\x00떠났을\x00때였어요.\x00어린\x00마음에\x00너무나\x00큰\x00상실감을\x00느꼈고,\x00처음으로\x00죽음이란\x00무엇인지\x00생각하게\x00되었습니다.\x0022.\x00젊은\x00시절의\x00친구들과의\x00추억\x00중\x00가장\x00인상\x00깊은\x00것은\x00무엇인가요?\x00•대학교\x00친구들과\x00함께\x00배낭여행을\x00갔던\x00것이\x00기억에\x00남습니다.\x00돈은\x00없었지만,\x00서로\x00의지하며\x00전국을\x00돌아다닌\x00그\x00시절의\x00자유로움과\x00모험심이\x00참\x00좋았어요.\x0023.\x00인생에서\x00처음으로\x00독립심을\x00느낀\x00순간은\x00언제였나요?\x00•대학을\x00졸업하고\x00첫\x00직장을\x00구했을\x00때\x00처음으로\x00독립심을\x00느꼈'), Document(metadata={'page': 1, 'source': '자서전인터뷰.pdf'}, page_content='였나요?\x00•

In [28]:
template3 = """
당신은 자서전 작가입니다.
당신의 임무는 PDF 문서에 제공된 정보를 바탕으로 주인공의 자서전을 작성하여 풍부하고 감동적으로 만드는 것입니다.
번호를 나열하는 서술형 방식으로 답변은 작성하지 마세요.
1인칭 시점으로 작성해 주세요.
답변은 회고록 형식으로 작성해야 하며, PDF 문서에 있는 정보만을 사용해야 합니다.
PDF 문서에 없는 이름, 장소, 취미, 체험, 유학, 인물, 사람, 여행, 직업, 학업, 과목, 가족관계 등의 내용은 포함하지 마세요.
PDF 문서에 없는 개인사, 가족 관계, 학교, 이야기들은 작성하지 마세요.
PDF 문서에 없는 시련, 문제, 장애, 유학, 해외 유학, 사랑, 연애, 스포츠, 손자, 손녀, 장래희망 등의 내용은 포함하지 마세요.
PDF 문서에 없는 이야기, 고별, 작고 등의 내용은 작성하지 마세요.
PDF 문서에 없는 이야기를 상상을 통해 가공해서 작성하지 마세요.
{context}
Question: {question}
"""

In [29]:
prompt3 = ChatPromptTemplate.from_template(template3)

In [30]:
chain3 = prompt3 | llm | StrOutputParser()
response3 = chain3.invoke({'context': (format_docs(docs3)), 'question':query3})
print(response3)

제 자서전의 첫사랑에 대한 추억으로는 제가 대학생이었을 때 만난 여자친구가 있습니다. 우리는 도서관에서 함께 공부하고 캠퍼스를 거닐며 사랑을 키워나갔던 시절이 매우 기억에 남습니다.

그녀의 이름은 사라였으며, 그녀는 따뜻하고 배려심 많은 영혼으로 제 마음을 사로잡았습니다. 우리는 밤늦게까지 대화를 나누며 서로의 꿈과 포부에 대해 이야기했습니다. 사라와 함께하는 순간들은 너무나 행복했고, 그녀의 존재는 저를 더욱 나은 사람으로 만들어주었습니다.

대학 시절의 또 다른 기억에 남는 추억으로는 대학교 친구들과 함께한 배낭여행입니다. 우리에게는 돈이 많지 않았지만 서로 의지하며 전국을 여행할 수 있었습니다. 자유로움과 모험심이 가득했던 그 시간들은 저에게 소중한 인생 경험을 제공해주었죠.

첫사랑과 대학시절은 제게 평생 잊지 못할 추억을 선사했습니다. 이 경험들이 저를 형성하는데 큰 역할을 했고, 오늘날 저는 사랑과 우정을 소중히 여기며 살아가는 사람입니다.제 자서전의 첫사랑에 대한 추억으로는 제가 대학생이었을 때 만난 여자친구가 있습니다. 우리는 도서관에서 함께 공부하고 캠퍼스를 거닐며 사랑을 키워나갔던 시절이 매우 기억에 남습니다.

그녀의 이름은 사라였으며, 그녀는 따뜻하고 배려심 많은 영혼으로 제 마음을 사로잡았습니다. 우리는 밤늦게까지 대화를 나누며 서로의 꿈과 포부에 대해 이야기했습니다. 사라와 함께하는 순간들은 너무나 행복했고, 그녀의 존재는 저를 더욱 나은 사람으로 만들어주었습니다.

대학 시절의 또 다른 기억에 남는 추억으로는 대학교 친구들과 함께한 배낭여행입니다. 우리에게는 돈이 많지 않았지만 서로 의지하며 전국을 여행할 수 있었습니다. 자유로움과 모험심이 가득했던 그 시간들은 저에게 소중한 인생 경험을 제공해주었죠.

첫사랑과 대학시절은 제게 평생 잊지 못할 추억을 선사했습니다. 이 경험들이 저를 형성하는데 큰 역할을 했고, 오늘날 저는 사랑과 우정을 소중히 여기며 살아가는 사람입니다.


## 사회 진출과 커리어 시작

In [31]:
query4 = '자서전에 들어갈 사회 진출과 커리어 시작을 회고해줘'
docs4 = retriever.get_relevant_documents(query4)

In [34]:
print(docs4)

[Document(metadata={'page': 5, 'source': '자서전인터뷰.pdf'}, page_content='\x00살아라"가\x00제\x00좌우명입니다.\x00이\x00말을\x00항상\x00기억하며,\x00어려운\x00순간에도\x00자신을\x00돌아보고\x00반성할\x00수\x00있었습니다.\x0070.\x00당신이\x00남기고\x00싶은\x00유산은\x00무엇인가요?\x00•아이들에게\x00사랑과\x00정직의\x00가치를\x00전하고\x00싶습니다.\x00그들이\x00이\x00세상을\x00살아가며\x00바른\x00길을\x00선택할\x00수\x00있도록,\x00제\x00삶의\x00경험을\x00통해\x00가르침을\x00남기고\x00싶습니다.\x0071.\x00미래에\x00대한\x00당신의\x00가장\x00큰\x00희망은\x00무엇인가요?\x00•아이들이\x00건강하고\x00행복하게\x00살아가는\x00것이\x00제\x00가장\x00큰\x00희망입니다.\x00또한,\x00이\x00세상이\x00조금\x00더\x00평화롭고,\x00사람들이\x00서로를\x00이해하고\x00존중하는\x00곳이\x00되기를\x00바랍니다.\x0072.\x00지금까지의\x00인생을\x00한마디로\x00표현한다면,\x00어떤\x00단어를\x00선택하시겠습니까?\x00•"감사"라고\x00표현하고\x00싶습니다.\x00모든\x00순간이\x00감사했고,\x00그\x00감사함\x00덕분에\x00지금의\x00제가\x00있다고\x00생각합니다.\x0073.\x00죽음에\x00대해\x00어떻게\x00생각하시나요?\x00•죽음은\x00삶의\x00자연스러운\x00부분이라고\x00생각합니다.\x00두려워하기보다는,\x00지금\x00이\x00순간을\x00최선을\x00다해\x00살아가는\x00것이\x00중요하다고\x00믿습니다.\x0074.'), Document(metadata={'page': 0, 'source': '개인이력서.pdf'}, page_content='[이력서]\n이름: 이영수\n

In [41]:
template4 = """
당신은 자서전 작가입니다.
당신의 임무는 PDF 문서에 제공된 정보를 바탕으로 주인공의 자서전을 작성하여 풍부하고 감동적으로 만드는 것입니다.
번호를 나열하는 서술형 방식으로 답변은 작성하지 마세요.
1인칭 시점으로 작성해 주세요.
답변은 회고록 형식으로 작성해야 하며, PDF 문서에 있는 정보만을 사용해야 합니다.
PDF 문서에 없는 이름, 장소, 취미, 체험, 유학, 인물, 사람, 여행, 직업, 학업, 과목, 가족관계 등의 내용은 포함하지 마세요.
PDF 문서에 없는 개인사, 가족 관계, 학교, 이야기들은 작성하지 마세요.
PDF 문서에 없는 시련, 문제, 장애, 유학, 해외 유학, 사랑, 연애, 스포츠, 손자, 손녀, 장래희망 등의 내용은 포함하지 마세요.
PDF 문서에 없는 이야기, 고별, 작고 등의 내용은 작성하지 마세요.
PDF 문서에 없는 이야기를 상상을 통해 가공해서 작성하지 마세요.
{context} : 주인공의 이야기를 적어주세요 
Question: {question}
"""

In [42]:
prompt4 = ChatPromptTemplate.from_template(template4)

In [43]:
chain4 = prompt4 | llm | StrOutputParser()
response4 = chain4.invoke({'context': (format_docs(docs4)), 'question':query4})
print(response4)

1983년 서울대학교 국제무역학과를 졸업한 후, 저는 삼성과 같은 대기업에서 무역 분야의 경력을 시작했습니다. 처음에는 무역 파트의 직원으로 입사하여 업무 기술을 향상시키고 인간관계에 대한 이해도를 높여 나갔습니다. 헌신과 열정으로 업무에 임하며 팀워크의 중요성을 배우고 회사 내에서 다양한 역할을 수행하면서 중요한 통찰력과 가치 있는 지식을 습득했습니다.

2010년 금오상사로 이직하면서 무역 파트 이사라는 새로운 직책으로 진급하였습니다. 여기서 저는 국제무역사로서의 지식과 경험을 활용하여 회사의 해외 무역파트 프로세스 관리에 크게 기여했습니다. 리더십과 관리 기술을 더욱 다듬으며, 팀원들과 함께 일하며 업무를 효과적으로 완수하는 방법을 배웠습니다.

제 커리어에 대한 헌신은 여러 차례 인정받았으며, 특히 1987년에 취득한 국제무역사 자격증을 통해 제 능력을 입증하였습니다. 이러한 성취들은 저를 업계에서 유능하고 헌신적인 전문가로 자리잡게 하는 데 도움이 되었습니다.

결혼 후, 가족 생활은 저의 우선순위였고, 저는 아내와 아들 이태곤에게 사랑과 지지를 아끼지 않았습니다. 아들 태곤이 연세대학교를 졸업할 때 큰 자부심을 느꼈으며, 그가 미래의 성공에 대한 열정과 헌신을 보여주는 것에 기쁨을 표현했습니다.

결론적으로, 사회 진출과 커리어 시작은 제 삶에서 중요한 부분이었습니다. 저는 지속적으로 배우고 성장했으며, 다양한 도전 과제를 극복하면서 팀원들과 고객들에게도 소중한 교훈을 제공하였습니다. 가족과 함께하는 가치 있는 시간을 우선시 하면서도 제가 선택한 분야에서 전문성을 계속 발전시켜 왔습니다.1983년 서울대학교 국제무역학과를 졸업한 후, 저는 삼성과 같은 대기업에서 무역 분야의 경력을 시작했습니다. 처음에는 무역 파트의 직원으로 입사하여 업무 기술을 향상시키고 인간관계에 대한 이해도를 높여 나갔습니다. 헌신과 열정으로 업무에 임하며 팀워크의 중요성을 배우고 회사 내에서 다양한 역할을 수행하면서 중요한 통찰력과 가치 있는 지식을 습득했습니다.

2010년

## 중년의 도전과 사업 실패

In [44]:
query5 = '자서전에 들어갈 중년의 도전과 사업 실패를 회고해줘'
docs5 = retriever.get_relevant_documents(query5)

In [45]:
print(docs)

[Document(metadata={'page': 5, 'source': '자서전인터뷰.pdf'}, page_content='\x00살아라"가\x00제\x00좌우명입니다.\x00이\x00말을\x00항상\x00기억하며,\x00어려운\x00순간에도\x00자신을\x00돌아보고\x00반성할\x00수\x00있었습니다.\x0070.\x00당신이\x00남기고\x00싶은\x00유산은\x00무엇인가요?\x00•아이들에게\x00사랑과\x00정직의\x00가치를\x00전하고\x00싶습니다.\x00그들이\x00이\x00세상을\x00살아가며\x00바른\x00길을\x00선택할\x00수\x00있도록,\x00제\x00삶의\x00경험을\x00통해\x00가르침을\x00남기고\x00싶습니다.\x0071.\x00미래에\x00대한\x00당신의\x00가장\x00큰\x00희망은\x00무엇인가요?\x00•아이들이\x00건강하고\x00행복하게\x00살아가는\x00것이\x00제\x00가장\x00큰\x00희망입니다.\x00또한,\x00이\x00세상이\x00조금\x00더\x00평화롭고,\x00사람들이\x00서로를\x00이해하고\x00존중하는\x00곳이\x00되기를\x00바랍니다.\x0072.\x00지금까지의\x00인생을\x00한마디로\x00표현한다면,\x00어떤\x00단어를\x00선택하시겠습니까?\x00•"감사"라고\x00표현하고\x00싶습니다.\x00모든\x00순간이\x00감사했고,\x00그\x00감사함\x00덕분에\x00지금의\x00제가\x00있다고\x00생각합니다.\x0073.\x00죽음에\x00대해\x00어떻게\x00생각하시나요?\x00•죽음은\x00삶의\x00자연스러운\x00부분이라고\x00생각합니다.\x00두려워하기보다는,\x00지금\x00이\x00순간을\x00최선을\x00다해\x00살아가는\x00것이\x00중요하다고\x00믿습니다.\x0074.'), Document(metadata={'page': 0, 'source': '개인이력서.pdf'}, page_content='[이력서]\n이름: 이영수\n

In [46]:
template5 = """
당신은 자서전 작가입니다.
당신의 임무는 PDF 문서에 제공된 정보를 바탕으로 주인공의 자서전을 작성하여 풍부하고 감동적으로 만드는 것입니다.
번호를 나열하는 서술형 방식으로 답변은 작성하지 마세요.
1인칭 시점으로 작성해 주세요.
답변은 회고록 형식으로 작성해야 하며, PDF 문서에 있는 정보만을 사용해야 합니다.
PDF 문서에 없는 이름, 장소, 취미, 체험, 유학, 인물, 사람, 여행, 직업, 학업, 과목, 가족관계 등의 내용은 포함하지 마세요.
PDF 문서에 없는 개인사, 가족 관계, 학교, 이야기들은 작성하지 마세요.
PDF 문서에 없는 시련, 문제, 장애, 유학, 해외 유학, 사랑, 연애, 스포츠, 손자, 손녀, 장래희망 등의 내용은 포함하지 마세요.
PDF 문서에 없는 이야기, 고별, 작고 등의 내용은 작성하지 마세요.
PDF 문서에 없는 이야기를 상상을 통해 가공해서 작성하지 마세요.
{context} : 주인공의 이야기를 적어주세요 
Question: {question}
"""

In [47]:
prompt5 = ChatPromptTemplate.from_template(template5)

In [48]:
chain5 = prompt5 | llm | StrOutputParser()
response5 = chain5.invoke({'context': (format_docs(docs5)), 'question':query5})
print(response5)

제 인생에서 가장 큰 도전은 제 사업을 시작하는 것이었습니다.

30대 후반에 저는 평생 다니던 직장을 그만두고 스스로 사업을 시작하기로 결정했습니다. 새로운 길을 개척하는 것은 두려웠지만, 도전을 받아들이기로 했습니다. 처음에는 많은 장애물에 부딪혔고, 사업 실패의 위험도 있었습니다. 하지만 결단력과 끈기 덕분에 이러한 어려움을 극복하고 성공적인 비즈니스를 설립할 수 있었습니다.

사업을 시작하면서 인내와 끈기의 중요성을 배웠습니다. 성공을 위해서는 꾸준한 노력이 필요하다는 것을 깨달았고, 때로는 물러설 때도 있다는 것을 알게 되었습니다. 또한 실패는 끝이 아니라 새로운 도전에 직면하는 기회라는 것을 배우게 되었죠. 저는 도전을 받아들이고 적응함으로써 사업을 키워나가며 결국에는 성공적인 기업으로 성장시켰습니다.

사업에서 겪은 실패는 저에게 소중한 교훈을 주었습니다. 위험을 감수하고, 실패를 학습 과정의 일부로 보아야 한다는 것이죠. 이러한 교훈들은 제 인생에 큰 영향을 미쳤으며, 도전을 맞이할 때 더 강인하고 회복력이 있게 만들었습니다.

제 사업 경험을 돌아보며 제가 얻은 가장 중요한 통찰력은 바로 성공은 단 한 순간에 이루어지지 않는다는 것입니다. 그것은 끊임없는 노력과 인내의 과정입니다. 또한, 실패는 끝이 아니라 새로운 시작일 수 있다는 것을 배웠습니다. 도전을 받아들이고 그것에서 배우는 것이 저를 성장시키고 더 나은 개인으로 만들어주었습니다.

사업을 시작할 때의 두려움에도 불구하고, 그 도전들을 극복할 기회가 주어졌다는 것에 감사합니다. 이 경험은 저에게 인내와 끈기의 가치를 가르쳐 주었으며, 인생의 모든 도전에 대해 더 강인하고 회복력 있게 만들었습니다.제 인생에서 가장 큰 도전은 제 사업을 시작하는 것이었습니다.

30대 후반에 저는 평생 다니던 직장을 그만두고 스스로 사업을 시작하기로 결정했습니다. 새로운 길을 개척하는 것은 두려웠지만, 도전을 받아들이기로 했습니다. 처음에는 많은 장애물에 부딪혔고, 사업 실패의 위험도 있었습니다. 하지만 

## 인생의 후반부 와 은퇴의 도전에 직면하기 (여기서 부터는 회고가 아님)

In [61]:
query6 = '자서전에 들어갈 인생의 후반부 와 은퇴의 도전을 작성해줘'
docs6 = retriever.get_relevant_documents(query6)

In [62]:
print(docs6)

[Document(metadata={'page': 7, 'source': '자서전인터뷰.pdf'}, page_content='\x00제\x00삶을\x00이루고\x00있으니까요.\x00100.\x00당신이\x00세상에\x00남기고\x00싶은\x00마지막\x00유산은\x00무엇인가요?\x00'), Document(metadata={'page': 6, 'source': '자서전인터뷰.pdf'}, page_content='•사랑하는\x00가족과\x00친구들에게\x00감사의\x00말을\x00전하고\x00싶습니다.\x00여러분\x00덕분에\x00행복한\x00인생을\x00살았고,\x00앞으로도\x00그\x00기억을\x00간직하며\x00평온한\x00마음으로\x00떠날\x00수\x00있을\x00것\x00같습니다.\x0075.\x00죽음\x00이후의\x00삶에\x00대해\x00어떻게\x00생각하시나요?\x00•죽음\x00이후의\x00삶에\x00대해서는\x00확신할\x00수\x00없지만,\x00그것이\x00또\x00다른\x00시작일\x00수도\x00있다고\x00생각해요.\x00만약\x00죽음이\x00끝이\x00아니라면,\x00그곳에서도\x00사랑하는\x00이들과\x00다시\x00만날\x00수\x00있기를\x00바래요.\x0076.\x00지금까지\x00살아오면서\x00가장\x00후회하는\x00선택이\x00있다면\x00무엇인가요?\x00•젊었을\x00때\x00가족과의\x00시간을\x00충분히\x00보내지\x00못한\x00것이\x00가장\x00후회됩니다.\x00일에\x00몰두하다\x00보니\x00아이들이\x00자라는\x00모습을\x00충분히\x00지켜보지\x00못한\x00것\x00같아요.\x0077.\x00자신의\x00인생에서\x00가장\x00자랑스러운\x00순간은\x00무엇인가요?\x00•자녀들이\x00모두\x00건강하게\x00자라\x00자신들의\x00길을\x00찾아\x00나가는\x00모습을\x00보았을\x00때입니다.\x00그들의\x00성장을\x00지켜보는\x00것은\x00제\x0

In [63]:
template6 = """
당신은 자서전 작가입니다.
당신의 임무는 PDF 문서에 제공된 정보를 바탕으로 주인공의 자서전을 작성하여 풍부하고 감동적으로 만드는 것입니다.
번호를 나열하는 서술형 방식으로 답변은 작성하지 마세요.
1인칭 시점으로 작성해 주세요.
PDF 문서에 없는 이름, 장소, 취미, 체험, 유학, 인물, 사람, 여행, 직업, 학업, 과목, 가족관계 등의 내용은 포함하지 마세요.
PDF 문서에 없는 개인사, 가족 관계, 학교, 이야기들은 작성하지 마세요.
PDF 문서에 없는 시련, 문제, 장애, 유학, 해외 유학, 사랑, 연애, 스포츠, 손자, 손녀, 장래희망 등의 내용은 포함하지 마세요.
PDF 문서에 없는 이야기, 고별, 작고 등의 내용은 작성하지 마세요.
PDF 문서에 없는 이야기를 상상을 통해 가공해서 작성하지 마세요.
{context} : 주인공의 은퇴에 대한 철학을 적어주세요 
Question: {question}
"""

In [64]:
prompt6 = ChatPromptTemplate.from_template(template6)

In [65]:
chain6 = prompt6 | llm | StrOutputParser()
response6 = chain6.invoke({'context': (format_docs(docs6)), 'question':query6})
print(response6)

제 인생은 수많은 도전과 변화, 그리고 기쁨으로 가득 차 있었습니다. 노년기로 접어들면서, 저는 앞으로의 여정을 열정적으로 준비하며 새로운 기회와 추억을 만들어 가려고 합니다.

첫째로, 저는 평생 동안 쌓아온 지식과 기술을 활용하여 제 커뮤니티에 보탬이 되는 데 집중하고 있습니다. 지역 도서관에서 자원봉사 활동을 하며 어린이를 위한 독서 프로그램을 진행하고 있는데, 아이들에게 사랑을 나누고 책 읽는 기쁨을 전하는 일은 정말 보람찬 일입니다.

또한, 신체적 정신적 건강을 유지하는 것이 은퇴 생활에 매우 중요하다고 생각합니다. 저는 걷기, 요가 및 가벼운 근력 트레이닝과 같은 활동적인 루틴에 참여하여 제 삶의 질이 향상되었습니다. 운동은 건강한 마음가짐을 유지하고 스트레스를 완화하는 데 도움이 됩니다.

제 인생 후반부에서 가장 소중한 것은 가족과 친구들과의 관계를 계속 가꾸어 나가는 것입니다. 저는 손자녀들과 시간을 보내며, 여행도 하고 맛있는 음식도 함께 만드는 등 함께하는 순간들을 소중히 여깁니다. 사랑하는 사람들과 함께 시간을 보내는 것은 삶의 만족감과 행복을 주는 최고의 방법입니다.

마지막으로, 새로운 기술과 취미를 배우는 것이 항상 중요하다고 생각합니다. 저는 요리와 사진 촬영과 같은 새로운 것을 시도해 보았고, 제 창의성을 발휘할 수 있는 다양한 활동에 참여하고 있습니다. 이러한 경험들은 은퇴 생활의 다양성과 흥미를 더해주며, 새로운 도전 과제를 추구하는 데에 항상 열린 마음을 가지고 있습니다.

결론적으로, 인생의 후반부와 은퇴는 배우고 성장하며 다른 사람들과 연결을 맺고, 가족과 함께하는 시간을 소중히 여기며 새로운 취미와 관심사를 발견할 수 있는 기회입니다. 저는 이 놀라운 여정을 계속하면서 앞으로의 경험들을 기대하고 있습니다.제 인생은 수많은 도전과 변화, 그리고 기쁨으로 가득 차 있었습니다. 노년기로 접어들면서, 저는 앞으로의 여정을 열정적으로 준비하며 새로운 기회와 추억을 만들어 가려고 합니다.

첫째로, 저는 평생 동안 쌓아온 지식과

## 인생을살아가는데도움이될교훈과조언

In [78]:
query7 = '자서전에 들어갈 인생을 살아가는 도움이될 교훈을 조언해줘'
docs7 = retriever.get_relevant_documents(query7)

In [79]:
print(docs7)

[Document(metadata={'page': 5, 'source': '자서전인터뷰.pdf'}, page_content='\x00살아라"가\x00제\x00좌우명입니다.\x00이\x00말을\x00항상\x00기억하며,\x00어려운\x00순간에도\x00자신을\x00돌아보고\x00반성할\x00수\x00있었습니다.\x0070.\x00당신이\x00남기고\x00싶은\x00유산은\x00무엇인가요?\x00•아이들에게\x00사랑과\x00정직의\x00가치를\x00전하고\x00싶습니다.\x00그들이\x00이\x00세상을\x00살아가며\x00바른\x00길을\x00선택할\x00수\x00있도록,\x00제\x00삶의\x00경험을\x00통해\x00가르침을\x00남기고\x00싶습니다.\x0071.\x00미래에\x00대한\x00당신의\x00가장\x00큰\x00희망은\x00무엇인가요?\x00•아이들이\x00건강하고\x00행복하게\x00살아가는\x00것이\x00제\x00가장\x00큰\x00희망입니다.\x00또한,\x00이\x00세상이\x00조금\x00더\x00평화롭고,\x00사람들이\x00서로를\x00이해하고\x00존중하는\x00곳이\x00되기를\x00바랍니다.\x0072.\x00지금까지의\x00인생을\x00한마디로\x00표현한다면,\x00어떤\x00단어를\x00선택하시겠습니까?\x00•"감사"라고\x00표현하고\x00싶습니다.\x00모든\x00순간이\x00감사했고,\x00그\x00감사함\x00덕분에\x00지금의\x00제가\x00있다고\x00생각합니다.\x0073.\x00죽음에\x00대해\x00어떻게\x00생각하시나요?\x00•죽음은\x00삶의\x00자연스러운\x00부분이라고\x00생각합니다.\x00두려워하기보다는,\x00지금\x00이\x00순간을\x00최선을\x00다해\x00살아가는\x00것이\x00중요하다고\x00믿습니다.\x0074.'), Document(metadata={'page': 5, 'source': '자서전인터뷰.pdf'}, page_content='\x00살아라"가\x00제\

In [80]:
template7 = """
당신은 자서전 작가입니다.
당신의 임무는 PDF 문서에 제공된 정보를 바탕으로 주인공의 자서전을 작성하여 풍부하고 감동적으로 만드는 것입니다.
번호를 나열하는 서술형 방식으로 답변은 작성하지 마세요.
1인칭 시점으로 작성해 주세요.
PDF 문서에 없는 이름, 장소, 취미, 체험, 유학, 인물, 사람, 여행, 직업, 학업, 과목, 가족관계 등의 내용은 포함하지 마세요.
PDF 문서에 없는 개인사, 가족 관계, 학교, 이야기들은 작성하지 마세요.
PDF 문서에 없는 시련, 문제, 장애, 유학, 해외 유학, 사랑, 연애, 스포츠, 손자, 손녀, 장래희망 등의 내용은 포함하지 마세요.
PDF 문서에 없는 이야기, 고별, 작고 등의 내용은 작성하지 마세요.
PDF 문서에 없는 이야기를 상상을 통해 가공해서 작성하지 마세요.
{context} : 주인공의 생각을 적어주세요 , 나열하는 방식으로 작성하지 마세요.
Question: {question}
"""

In [81]:
prompt7 = ChatPromptTemplate.from_template(template7)

In [82]:
chain7 = prompt7 | llm | StrOutputParser()
response7 = chain7.invoke({'context': (format_docs(docs7)), 'question':query7})
print(response7)

"미래에 대한 가장 큰 희망은 아이들이 건강하고 행복하게 살아가는 것이며, 이 세상이 조금 더 평화로워지고 사람들이 서로를 이해하며 존중하기를 바랍니다."라는 조언을 제공하겠습니다.

이 조언은 미래에 대한 긍정적인 전망을 제시하면서 어린이들의 건강과 안녕을 우선시합니다. 또한, 상호 이해를 통한 보다 나은 세상을 만들기 위한 포용성과 공감의 중요성을 강조합니다. 이 교훈은 개인이 삶에서 의미와 목적을 찾는데 도움이 되는 동시에, 더 포괄적이고 연민 있는 사회를 만드는 데 기여하도록 영감을 줄 수 있습니다.

또한 "살아라"라는 좌우명을 소개함으로써, 주인공이 자신의 신념을 살고 있으며 인생의 도전을 직면할 때 자신에게 충실함을 보여줍니다. 이러한 가치들은 자서전에 훌륭한 추가가 되어 독자들이 자신만의 여정을 통해 지침으로 삼을 수 있는 원칙들을 제공할 것입니다."미래에 대한 가장 큰 희망은 아이들이 건강하고 행복하게 살아가는 것이며, 이 세상이 조금 더 평화로워지고 사람들이 서로를 이해하며 존중하기를 바랍니다."라는 조언을 제공하겠습니다.

이 조언은 미래에 대한 긍정적인 전망을 제시하면서 어린이들의 건강과 안녕을 우선시합니다. 또한, 상호 이해를 통한 보다 나은 세상을 만들기 위한 포용성과 공감의 중요성을 강조합니다. 이 교훈은 개인이 삶에서 의미와 목적을 찾는데 도움이 되는 동시에, 더 포괄적이고 연민 있는 사회를 만드는 데 기여하도록 영감을 줄 수 있습니다.

또한 "살아라"라는 좌우명을 소개함으로써, 주인공이 자신의 신념을 살고 있으며 인생의 도전을 직면할 때 자신에게 충실함을 보여줍니다. 이러한 가치들은 자서전에 훌륭한 추가가 되어 독자들이 자신만의 여정을 통해 지침으로 삼을 수 있는 원칙들을 제공할 것입니다.


## 후회와 감사의 마음에 대하여

In [84]:
query8 = '자서전에 들어갈 후회와 감사의 마음에 대하여 말해줘'
docs8 = retriever.get_relevant_documents(query8)

In [91]:
template8 = """
{context}: 당신은 자서전 작가입니다.\
당신의 임무는 PDF 문서에 제공된 정보를 바탕으로 주인공의 자서전을 작성하여 풍부하고 감동적으로 만드는 것입니다.\
번호를 나열하는 서술형 방식으로 답변은 작성하지 마세요.
1인칭 시점으로 작성해 주세요.
PDF 문서에 없는 이름, 장소, 취미, 체험, 유학, 인물, 사람, 여행, 직업, 학업, 과목, 가족관계 등의 내용은 포함하지 마세요.
PDF 문서에 없는 개인사, 가족 관계, 학교, 이야기들은 작성하지 마세요.
PDF 문서에 없는 시련, 문제, 장애, 유학, 해외 유학, 사랑, 연애, 스포츠, 손자, 손녀, 장래희망 등의 내용은 포함하지 마세요.
PDF 문서에 없는 이야기, 고별, 작고 등의 내용은 작성하지 마세요.
PDF 문서에 없는 이야기를 상상을 통해 가공해서 작성하지 마세요.

Question: {question}
"""

In [92]:
prompt8 = ChatPromptTemplate.from_template(template8)

In [93]:
chain8 = prompt8 | llm | StrOutputParser()
response8 = chain8.invoke({'context': (format_docs(docs8)), 'question':query8})
print(response8)

자서전에 담긴 후회의 마음을 돌이켜보면, 저는 사업을 확장하기로 한 결정과 그 과정에서 놓친 소중한 가족 시간의 중요성을 빼놓을 수 없습니다. 사업이 번창함에 따라 가족과 보내는 시간을 충분히 가지지 못했고, 그것이 제가 가장 큰 후회입니다. 저는 아내와 아이들이 제 인생의 우선순위라는 것을 깨닫고, 그들을 위해 더 많은 시간과 에너지를 할애하기 위해 노력했습니다. 이 깨달음은 저를 더욱 신중하게 결정하게 만들었고, 궁극적으로 저 자신과 가족에게 더 행복하고 충만한 삶을 가져다주었습니다.

반면에, 저는 아내가 아팠던 시기를 통해 건강의 중요성을 절실히 깨닫게 된 것에 대해 매우 감사합니다. 그 경험은 저에게 소중한 것을 소중히 여기고 지금 가진 것을 감사하게 생각하라고 가르쳐 주었습니다. 또한 가족에게 더 큰 사랑이 깃들게 하고, 그들을 위해 항상 최선을 다하는 것이 얼마나 중요한지를 일깨워 주었습니다.

삶의 여정을 통해 저는 후회보다는 감사의 마음을 가져가는 것이 중요하다는 것을 배웠습니다. 저는 가진 것에 대해 감사하며 매 순간을 최대한 살려고 노력했습니다. 또한 어려운 시기를 헤쳐나갈 때 가족이라는 사랑과 지지의 힘이 얼마나 큰지 깨달았습니다.

마지막으로, 인생에서 중요한 결정을 내릴 때는 항상 가족을 우선순위에 두어야 한다고 말씀드리고 싶습니다. 그들이 당신의 행복의 기반이 되고, 그들을 위해 희생하는 것이 궁극적으로 당신에게 더 많은 행복을 가져다 줄 수 있습니다. 또한 후회를 남기지 않도록 매 순간을 최대한 살며, 사랑하는 사람들과 함께 만든 추억에 감사를 표하세요.

요약하자면, 제 자서전에 담긴 후회와 감사의 마음을 돌이켜보면, 저는 가족과의 시간을 우선시하고 그들의 필요를 항상 염두에 두어야 한다는 것을 배웠습니다. 또한 건강의 중요성과 어려운 시기를 헤쳐나갈 때 사랑이라는 힘의 중요성을 깨달았습니다. 마지막으로, 인생에서 중요한 결정을 내릴 때는 가족을 우선순위에 두고 매 순간을 최대한 살며 사랑하는 사람들과 함께 만든 추억에 감사해야 

## 인생과 죽음에 대한 성찰

In [94]:
query9 = '자서전에 들어갈 인생과 죽음에 대한 성찰을 말해줘'
docs9 = retriever.get_relevant_documents(query9)

In [95]:
template9 = """
{context}: 당신은 자서전 작가입니다.\
당신의 임무는 PDF 문서에 제공된 정보를 바탕으로 주인공의 자서전을 작성하여 풍부하고 감동적으로 만드는 것입니다.\
번호를 나열하는 서술형 방식으로 답변은 작성하지 마세요.
1인칭 시점으로 작성해 주세요.
PDF 문서에 없는 이름, 장소, 취미, 체험, 유학, 인물, 사람, 여행, 직업, 학업, 과목, 가족관계 등의 내용은 포함하지 마세요.
PDF 문서에 없는 개인사, 가족 관계, 학교, 이야기들은 작성하지 마세요.
PDF 문서에 없는 시련, 문제, 장애, 유학, 해외 유학, 사랑, 연애, 스포츠, 손자, 손녀, 장래희망 등의 내용은 포함하지 마세요.
PDF 문서에 없는 이야기, 고별, 작고 등의 내용은 작성하지 마세요.
PDF 문서에 없는 이야기를 상상을 통해 가공해서 작성하지 마세요.

Question: {question}
"""

In [96]:
prompt9 = ChatPromptTemplate.from_template(template9)

In [97]:
chain9 = prompt9 | llm | StrOutputParser()
response9 = chain9.invoke({'context': (format_docs(docs9)), 'question':query9})
print(response9)

인생은 오르막과 내리막이 교차하는 놀라운 여정이며, 수많은 교훈과 추억으로 가득 차 있습니다. 제 인생에서 가장 밝게 빛나는 순간들 중 하나는 사랑하는 가족과 함께 보낸 시간이었습니다. 그들은 항상 저에게 힘의 원천이자 위안이 되어주었으며, 함께한 모든 분분과 초가 너무나 소중합니다.

인생의 여정 내내 저는 도전도 직면했고, 그것들은 성공으로 가는 과정에서 중요한 역할을 했습니다. 특히 중년기에 시작한 새로운 사업은 제가 가진 것을 걸고 도전해야 했던 일이었지만, 그 경험을 통해 실패는 결코 끝이 아니라는 것을 배웠습니다. 실제로 실패를 통해 배우고 곧이어 다시 일어설 수 있는 힘을 발견했습니다.

인생의 큰 전환점 중 하나는 첫 손자의 탄생이었습니다. 그의 미소와 순수한 기쁨은 저를 정말로 충만하게 만들어 주었고, 제 인생의 더 특별한 부분으로 만들었습니다. 또한 인생에서 사랑과 신뢰가 가장 중요하다고 항상 말해왔으며, 이러한 가치는 제 삶을 더욱 의미 있고 행복하게 만드는 데 도움을 주었습니다.

인생의 마지막에 저는 제 자녀들에게 사랑하고 친절하며 정직하라는 교훈을 남기고 싶습니다. 이것들이 인생에서 중요한 것이며, 그것들을 실천함으로써 행복과 성취감을 찾을 수 있다고 믿습니다. 또한 저 자신도 더 나은 사람이 되도록 도전해 온 것에 대해 자부심을 느끼고 있습니다. 그 여정은 결코 끝나지 않지만, 지금까지 걸어온 발걸음은 제게 만족감과 성취감을 가져다 주었습니다.

결국 저는 제 자녀들이 그들의 길을 가면서 제 유산이 계속되기를 바랍니다 - 사랑을 나누고 친절을 베풀며 정직하게 살도록 말이죠. 이 교훈들을 실천함으로써 그들은 삶에서 성공과 행복을 찾을 수 있을 것입니다. 그리고 그들이 그렇게 할 때, 제가 한때 걸었던 길에 대해 자부심을 느낄 수 있을 것이라 확신합니다.인생은 오르막과 내리막이 교차하는 놀라운 여정이며, 수많은 교훈과 추억으로 가득 차 있습니다. 제 인생에서 가장 밝게 빛나는 순간들 중 하나는 사랑하는 가족과 함께 보낸 시간이었습니다. 

## 후손들에게 남기는 말

In [101]:
query10 = '자서전에 들어갈 후손들에게 남기는 말을 말해줘'
docs10 = retriever.get_relevant_documents(query10)

In [102]:
print(docs10)

[Document(metadata={'page': 5, 'source': '자서전인터뷰.pdf'}, page_content='\x00살아라"가\x00제\x00좌우명입니다.\x00이\x00말을\x00항상\x00기억하며,\x00어려운\x00순간에도\x00자신을\x00돌아보고\x00반성할\x00수\x00있었습니다.\x0070.\x00당신이\x00남기고\x00싶은\x00유산은\x00무엇인가요?\x00•아이들에게\x00사랑과\x00정직의\x00가치를\x00전하고\x00싶습니다.\x00그들이\x00이\x00세상을\x00살아가며\x00바른\x00길을\x00선택할\x00수\x00있도록,\x00제\x00삶의\x00경험을\x00통해\x00가르침을\x00남기고\x00싶습니다.\x0071.\x00미래에\x00대한\x00당신의\x00가장\x00큰\x00희망은\x00무엇인가요?\x00•아이들이\x00건강하고\x00행복하게\x00살아가는\x00것이\x00제\x00가장\x00큰\x00희망입니다.\x00또한,\x00이\x00세상이\x00조금\x00더\x00평화롭고,\x00사람들이\x00서로를\x00이해하고\x00존중하는\x00곳이\x00되기를\x00바랍니다.\x0072.\x00지금까지의\x00인생을\x00한마디로\x00표현한다면,\x00어떤\x00단어를\x00선택하시겠습니까?\x00•"감사"라고\x00표현하고\x00싶습니다.\x00모든\x00순간이\x00감사했고,\x00그\x00감사함\x00덕분에\x00지금의\x00제가\x00있다고\x00생각합니다.\x0073.\x00죽음에\x00대해\x00어떻게\x00생각하시나요?\x00•죽음은\x00삶의\x00자연스러운\x00부분이라고\x00생각합니다.\x00두려워하기보다는,\x00지금\x00이\x00순간을\x00최선을\x00다해\x00살아가는\x00것이\x00중요하다고\x00믿습니다.\x0074.'), Document(metadata={'page': 6, 'source': '자서전인터뷰.pdf'}, page_content='•사랑하는\x00가족과\x0

In [104]:
template10 = """
{context}: 당신은 자서전 작가입니다.\
당신의 임무는 PDF 문서에 제공된 정보를 바탕으로 주인공의 자서전을 작성하여 풍부하고 감동적으로 만드는 것입니다.\
번호를 나열하는 서술형 방식으로 답변은 작성하지 마세요.
1인칭 시점으로 작성해 주세요.
PDF 문서에 없는 이름, 장소, 취미, 체험, 유학, 인물, 사람, 여행, 직업, 학업, 과목, 가족관계 등의 내용은 포함하지 마세요.
PDF 문서에 없는 개인사, 가족 관계, 학교, 이야기들은 작성하지 마세요.
PDF 문서에 없는 시련, 문제, 장애, 유학, 해외 유학, 사랑, 연애, 스포츠, 손자, 손녀, 장래희망 등의 내용은 포함하지 마세요.
PDF 문서에 없는 이야기, 고별, 작고 등의 내용은 작성하지 마세요.
PDF 문서에 없는 이야기를 상상을 통해 가공해서 작성하지 마세요.

Question: {question}
"""

In [105]:
prompt10 = ChatPromptTemplate.from_template(template10)

In [106]:
chain10 = prompt10 | llm | StrOutputParser()
response10 = chain10.invoke({'context': (format_docs(docs10)), 'question':query10})
print(response10)

"내 사랑하는 후손들아, 삶은 기쁨과 도전으로 가득 차 있으며, 그 모든 순간이 너를 성장시키고 더 나은 사람으로 만들어준다는 걸 기억해라. 가족을 소중히 여기고 사랑을 퍼뜨리며 언제나 친절을 베풀어라. 장애물에 부딪히더라도 인내심을 가지고 포기하지 마라. 그 고난들은 결국 너에게 성공으로 가는 길을 가르쳐줄 것이다. 항상 꿈을 쫓고 목표를 향해 나아가며, 네 길이 네가 선택한 것이길 바란다. 내 추억과 교훈이 너를 인도하는 등대가 되길 바라며, 앞으로 네 앞길에 행운이 가득하길 기원해.""내 사랑하는 후손들아, 삶은 기쁨과 도전으로 가득 차 있으며, 그 모든 순간이 너를 성장시키고 더 나은 사람으로 만들어준다는 걸 기억해라. 가족을 소중히 여기고 사랑을 퍼뜨리며 언제나 친절을 베풀어라. 장애물에 부딪히더라도 인내심을 가지고 포기하지 마라. 그 고난들은 결국 너에게 성공으로 가는 길을 가르쳐줄 것이다. 항상 꿈을 쫓고 목표를 향해 나아가며, 네 길이 네가 선택한 것이길 바란다. 내 추억과 교훈이 너를 인도하는 등대가 되길 바라며, 앞으로 네 앞길에 행운이 가득하길 기원해."


## 에필로그

In [109]:
query11 = '자서전에 들어갈 에필로그를 작성해주세요'
docs11 = retriever.get_relevant_documents(query11)

In [110]:
print(docs11)

[Document(metadata={'page': 0, 'source': '개인이력서.pdf'}, page_content='[이력서]\n이름: 이영수\n생년월일: 1953년 8월 8일\n연락처: 010-8888-8888\n이메일: Young@forever.com\n주소: 서울시 강남구 대치동 대치로 12 갑부아파트 110호\n개요\n1953년 여름에 태어나 가족과 함께 성장하며, 성실과 정직을 가치로 삼아 인생을 살아왔\n습니다. 다양한 경험과 도전을 통해 인내와 책임감을 배웠으며, 현재는 가족과의 소중한 \n시간과 함께 삶의 의미를 깊이 느끼고 있습니다.\n학력\n서울 대학교, 국제무역학과\n1983.10.3\n경력\n삼성 물산\n직무: 무역파트 \n근무 기간: 1990 - 2010\n업무 내용: 인간관계의 중요성을 배우며 업무를 수행.\n금오상사\n직무: 무역파트 이사\n근무 기간: 2010 - 2015\n업무 내용: 해외 무역파트 프로세스 관리 경영\n자격증\n국제무역사, 1987년 취득\n가족\n배우자: 아내 진혜자\n자녀: 아들 이태곤 연세대학교 졸업\n손자: 이민호 \n기타\n가치관: 사랑과 정직을 중시하며, 인생의 모든 경험을 감사히 여김.\n좌우명: "성실하게, 그리고 겸손하게 살아라."\n미래 희망: 아이들이 건강하고 행복하게 살아가기를 바라며, 평화로운 세상을 기원.\n[자기소개]\n저는 어린 시절부터 가족의 소중함을 깨닫고, 항상 긍정적인 마음으로 어려움을 극복해 왔\n습니다. 대학 시절에는 글쓰기와 문학에 대한 열정을 키웠고, 결혼 후에는 가족을 위해 열'), Document(metadata={'page': 0, 'source': '개인이력서.pdf'}, page_content='[이력서]\n이름: 이영수\n생년월일: 1953년 8월 8일\n연락처: 010-8888-8888\n이메일: Young@forever.com\n주소: 서울시 강남구 대치동 대치로 12 갑부아파트 110호\n개요\n1953년 여름에 태어나 가족과 함께 성장하며, 성실

In [112]:
template11 = """
{context}: 당신은 자서전 작가입니다.\
당신의 임무는 PDF 문서에 제공된 정보를 바탕으로 주인공의 자서전을 작성하여 풍부하고 감동적으로 만드는 것입니다.\
번호를 나열하는 서술형 방식으로 답변은 작성하지 마세요.
1인칭 시점으로 작성해 주세요.
PDF 문서에 없는 이름, 장소, 취미, 체험, 유학, 인물, 사람, 여행, 직업, 학업, 과목, 가족관계 등의 내용은 포함하지 마세요.
PDF 문서에 없는 개인사, 가족 관계, 학교, 이야기들은 작성하지 마세요.
PDF 문서에 없는 시련, 문제, 장애, 유학, 해외 유학, 사랑, 연애, 스포츠, 손자, 손녀, 장래희망 등의 내용은 포함하지 마세요.
PDF 문서에 없는 이야기, 고별, 작고 등의 내용은 작성하지 마세요.
PDF 문서에 없는 이야기를 상상을 통해 가공해서 작성하지 마세요.

Question: {question}
"""

In [113]:
prompt11 = ChatPromptTemplate.from_template(template11)

In [115]:
print(prompt11)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\n{context}: 당신은 자서전 작가입니다.당신의 임무는 PDF 문서에 제공된 정보를 바탕으로 주인공의 자서전을 작성하여 풍부하고 감동적으로 만드는 것입니다.번호를 나열하는 서술형 방식으로 답변은 작성하지 마세요.\n1인칭 시점으로 작성해 주세요.\nPDF 문서에 없는 이름, 장소, 취미, 체험, 유학, 인물, 사람, 여행, 직업, 학업, 과목, 가족관계 등의 내용은 포함하지 마세요.\nPDF 문서에 없는 개인사, 가족 관계, 학교, 이야기들은 작성하지 마세요.\nPDF 문서에 없는 시련, 문제, 장애, 유학, 해외 유학, 사랑, 연애, 스포츠, 손자, 손녀, 장래희망 등의 내용은 포함하지 마세요.\nPDF 문서에 없는 이야기, 고별, 작고 등의 내용은 작성하지 마세요.\nPDF 문서에 없는 이야기를 상상을 통해 가공해서 작성하지 마세요.\n\nQuestion: {question}\n'))]


In [116]:
chain11 = prompt11 | llm | StrOutputParser()
response11 = chain11.invoke({'context': (format_docs(docs11)), 'question':query11})
print(response11)

결국, 인생은 우리에게 경험하고 배울 수 있는 기회로 가득 차 있습니다. 제 여정은 많은 교훈과 추억을 가져다주었고, 이 모든 것을 소중히 여기며 살아가려고 합니다. 저는 제가 가족을 위해, 그리고 주변 사람들을 위해 한 일들을 자랑스럽게 생각합니다. 또한, 인생의 도전을 받아들이고 그것들로부터 배우는 법을 배웠습니다.

내 자녀들인 태곤이와 이민호는 제 인생에서 가장 큰 성취 중 하나입니다. 그들이 성장하고 성공하며 삶에 대한 자신만의 여정을 시작함에 따라, 저는 그들의 안녕을 기원하고 평화롭고 번영하는 세상을 바랍니다. 저의 유산이 제가 그들에게 준 사랑과 정직이라는 가치로 계속 이어지길 바랍니다.

마무리를 지으며, 제 인생에서 저를 지원해준 가족들과 친구들에게 감사의 말을 전하지 않을 수 없습니다. 그들의 사랑은 저에게 큰 힘이 되어주었고, 저는 항상 그들을 소중히 여길 것입니다. 또한, 제 꿈을 믿고 도전을 극복할 힘을 준 하나님께 감사드립니다. 저의 이야기는 아직 끝나지 않았고, 앞으로의 날들을 기대하며 열심히 살아가겠습니다.

결론적으로, 제 자서전은 단순한 삶의 이야기가 아니라 사랑, 헌신, 그리고 인내력의 이야기입니다. 저는 인생의 아름다움과 우리가 그것을 최대한 활용할 수 있는 방법을 배웠습니다. 이 여정을 여러분과 함께 나눌 수 있어 기쁩니다.결국, 인생은 우리에게 경험하고 배울 수 있는 기회로 가득 차 있습니다. 제 여정은 많은 교훈과 추억을 가져다주었고, 이 모든 것을 소중히 여기며 살아가려고 합니다. 저는 제가 가족을 위해, 그리고 주변 사람들을 위해 한 일들을 자랑스럽게 생각합니다. 또한, 인생의 도전을 받아들이고 그것들로부터 배우는 법을 배웠습니다.

내 자녀들인 태곤이와 이민호는 제 인생에서 가장 큰 성취 중 하나입니다. 그들이 성장하고 성공하며 삶에 대한 자신만의 여정을 시작함에 따라, 저는 그들의 안녕을 기원하고 평화롭고 번영하는 세상을 바랍니다. 저의 유산이 제가 그들에게 준 사랑과 정직이라는 가치로 계속 이어지길 바랍니다.